Input

In [9]:
file_path = "input.txt"  # Path to the input file

with open(file_path, "r") as f:
    data = f.read()

inps, ops = data.split('\n\n')
inps = {z.split(':')[0]: int(z.split(':')[1].strip()) for z in inps.split('\n')}

ops = [op.split(' ') for op in ops.split('\n')]

Part 1

In [6]:
import collections

def solve_gate(inps, ops):
    not_visited = set([i for i in range(len(ops))])
    while len(not_visited):
        for v in not_visited:
            elem = ops[v]
            if elem[0] in inps and elem[2] in inps:
                op = elem[1]
                if op == 'AND':
                    inps[elem[4]] = inps[elem[0]] and inps[elem[2]]
                elif op == 'XOR':
                    inps[elem[4]] = inps[elem[0]] ^ inps[elem[2]]
                else:
                    inps[elem[4]] = inps[elem[0]] or inps[elem[2]]
                not_visited.remove(v)
                break
    od = collections.OrderedDict(sorted(inps.items()))
    rep = ''
    for k in od:
        if k.startswith('z'): rep += str(inps[k])
    return int(rep[::-1], 2)

In [7]:
solve_gate(inps, ops)

57344080719736

alternative solution to part 1 using z3

In [10]:
from z3 import *

solver = Solver()

z3_vars = {}

for var, value in inps.items():
    z3_vars[var] = Bool(var) if value is None else BoolVal(value)

for op in ops:
    input1 = z3_vars[op[0]] if op[0] in z3_vars else Bool(op[0])
    input2 = z3_vars[op[2]] if op[2] in z3_vars else Bool(op[2])
    output = op[4]
    
    if output not in z3_vars:
        z3_vars[output] = Bool(output)
    
    if op[1] == 'AND':
        solver.add(z3_vars[output] == And(input1, input2))
    elif op[1] == 'OR':
        solver.add(z3_vars[output] == Or(input1, input2))
    elif op[1] == 'XOR':
        solver.add(z3_vars[output] == Xor(input1, input2))

if solver.check() == sat:
    model = solver.model()

    z_values = [model[z3_vars[f'z{i:02}']] for i in range(46)]

    result = ''.join(['1' if z else '0' for z in reversed(z_values)])
    print("Final binary result:", result)
    print("Integer value:", int(result, 2))
else:
    print("No solution found")

Final binary result: 1101000010011101110101100011111010111101111000
Integer value: 57344080719736


Part 2: [Not solved yet]